<a href="https://colab.research.google.com/github/abdurahman02/AcademicContent/blob/master/FLMF03_Running.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import os
import copy
import time
import string

In [2]:
!git clone "https://github.com/abdurahman02/ml-latest-small.git"
os.chdir("ml-latest-small")
os.listdir()

fatal: destination path 'ml-latest-small' already exists and is not an empty directory.


['ml-latest-small',
 'links.csv',
 'tags.csv',
 'results_02.txt',
 'results.md',
 'ratings.csv',
 'README.md',
 'movies.csv',
 '.git']

In [4]:
data = pd.read_csv("ratings.csv")
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# split train and validation before encoding
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()

In [6]:
# here is a handy function modified from fast.ai
def proc_col(col, train_col=None):
    """Encodes a pandas column with continous ids. 
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["userId", "movieId"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [7]:
# encoding the train and validation data
df_train = encode_data(train)
df_val = encode_data(val, train)

In [8]:
num_users = len(df_train.userId.unique())
num_items = len(df_train.movieId.unique())
print(num_users, num_items)

610 8998


In [9]:
def add_model_parameters(model1, model2):
    # Adds the parameters of model1 to model2

    params1 = model1.named_parameters()
    params2 = model2.named_parameters()

    dict_params2 = dict(params2)

    for name1, param1 in params1:
        if name1 in dict_params2:
        #   if name1 != 'userX_embedding.weight':
            dict_params2[name1].data.copy_(param1.data + dict_params2[name1].data)

    model2.load_state_dict(dict_params2)

def sub_model_parameters(model1, model2):
    # Subtracts the parameters of model2 with model1

    params1 = model1.named_parameters()
    params2 = model2.named_parameters()

    dict_params2 = dict(params2)

    for name1, param1 in params1:
        if name1 in dict_params2:
        #   if name1 != 'userX_embedding.weight':
            dict_params2[name1].data.copy_(dict_params2[name1].data - param1.data)

    model2.load_state_dict(dict_params2)

def divide_model_parameters(model, f):
    # Divides model parameters except for the user embeddings with f
    params1 = model.named_parameters()
    params2 = model.named_parameters()
    dict_params2 = dict(params2)
    for name1, param1 in params1:
        if name1 != 'user_emb.weight':
            dict_params2[name1].data.copy_(param1.data / f)
    model.load_state_dict(dict_params2)

def zero_model_parameters(model):
    # sets all parameters to zero

    params1 = model.named_parameters()
    params2 = model.named_parameters()
    dict_params2 = dict(params2)
    for name1, param1 in params1:
        if name1 in dict_params2:
            dict_params2[name1].data.copy_(param1.data - dict_params2[name1].data)

    model.load_state_dict(dict_params2)

def AssignUserEmbedding(model, user_emb):
  params1 = model.named_parameters()
  dict_params1 = dict(params1)
  dict_params1['userX_embedding.weight'].data = copy.deepcopy(user_emb)
  model.load_state_dict(dict_params1)

def printModelParam(model):
  print("hello world")
  params1 = model.named_parameters()
  print(params1)
  dict_params1 = dict(params1)
  print(dict_params1)
  print(dict_params1['userX_embedding.weight'].data)
  # for name1, param1 in params1:
  #   print("hello world")
  #   print(name1)
  #   print(param1)


In [10]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)

In [34]:
class MF02(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF02, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        # self.user_emb.weight.data.uniform_(0, 0.05)
        # self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)

In [35]:
# if torch.cuda.is_available():
#   model = MF(num_users, num_items, emb_size=100).cuda()
# else:
#   model = MF(num_users, num_items, emb_size=100)

In [36]:
def test_loss(model01, unsqueeze=False):
    model01.eval()
    if torch.cuda.is_available():
      users = torch.LongTensor(df_val.userId.values).cuda()
      items = torch.LongTensor(df_val.movieId.values).cuda()
      ratings = torch.FloatTensor(df_val.rating.values).cuda()
    else:
      users = torch.LongTensor(df_val.userId.values)
      items = torch.LongTensor(df_val.movieId.values)
      ratings = torch.FloatTensor(df_val.rating.values)
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model01(users, items)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())

In [44]:
def train_epocs(model_server, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    
    model_diff = copy.deepcopy(model_server)
    if torch.cuda.is_available():
      model_diff.cuda()
    zero_model_parameters(model_diff)

    t1 = time.time()
    

    # optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    

    for user_id in df_train.userId.unique():

        model02 = copy.deepcopy(model_server)
        model02.train()
        if torch.cuda.is_available():
            model02.cuda()
        optimizer = torch.optim.Adam(model02.parameters(), lr=lr, weight_decay=wd)
    
        sel_rows_of_user_i = df_train[(df_train.userId == user_id)]
        if torch.cuda.is_available():
            users = torch.LongTensor(sel_rows_of_user_i.userId.values).cuda()
            items = torch.LongTensor(sel_rows_of_user_i.movieId.values).cuda()
            ratings = torch.FloatTensor(sel_rows_of_user_i.rating.values).cuda()
        else:
            users = torch.LongTensor(sel_rows_of_user_i.userId.values) # .cuda()
            items = torch.LongTensor(sel_rows_of_user_i.movieId.values) #.cuda()
            ratings = torch.FloatTensor(sel_rows_of_user_i.rating.values) #.cuda()
        if unsqueeze:
            ratings = ratings.unsqueeze(1)

        
        y_hat = model02(users, items)
        loss = F.mse_loss(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(loss.item())

        sub_model_parameters(model_server, model02)
        add_model_parameters(model02, model_diff)

    # Take the average of the MLP and item vectors
    divide_model_parameters(model_diff, (len(df_train.userId.unique())))

    # Update the global model by adding the total change
    add_model_parameters(model_diff, model_server)
    
    t2 = time.time()
    print("Time of round:", round(t2 - t1), "seconds")

    test_loss(model_server, unsqueeze)
    return model_server

In [45]:
if torch.cuda.is_available():
  model_s = MF(num_users, num_items, emb_size=100).cuda()
else:
  model_s = MF(num_users, num_items, emb_size=100)

In [46]:
for t in range(10):
    train_epocs(model_s, epochs=10, lr=0.1)

Time of round: 10 seconds
test loss 10.683 
Time of round: 11 seconds
test loss 7.804 
Time of round: 10 seconds
test loss 5.115 
Time of round: 10 seconds
test loss 3.367 
Time of round: 10 seconds
test loss 2.370 
Time of round: 10 seconds
test loss 1.901 
Time of round: 10 seconds
test loss 1.630 
Time of round: 10 seconds
test loss 1.481 
Time of round: 10 seconds
test loss 1.344 
Time of round: 10 seconds
test loss 1.261 


In [47]:
for t in range(10):
    train_epocs(model_s, epochs=15, lr=0.01)

Time of round: 10 seconds
test loss 1.227 
Time of round: 10 seconds
test loss 1.199 
Time of round: 10 seconds
test loss 1.176 
Time of round: 10 seconds
test loss 1.158 
Time of round: 10 seconds
test loss 1.144 
Time of round: 10 seconds
test loss 1.133 
Time of round: 10 seconds
test loss 1.124 
Time of round: 10 seconds
test loss 1.115 
Time of round: 10 seconds
test loss 1.107 
Time of round: 10 seconds
test loss 1.100 


In [48]:
for t in range(30):
    train_epocs(model_s, epochs=15, lr=0.01)

Time of round: 10 seconds
test loss 1.093 
Time of round: 10 seconds
test loss 1.086 
Time of round: 10 seconds
test loss 1.080 
Time of round: 10 seconds
test loss 1.074 
Time of round: 10 seconds
test loss 1.067 
Time of round: 10 seconds
test loss 1.062 
Time of round: 10 seconds
test loss 1.056 
Time of round: 10 seconds
test loss 1.051 
Time of round: 11 seconds
test loss 1.045 
Time of round: 10 seconds
test loss 1.040 
Time of round: 10 seconds
test loss 1.035 
Time of round: 10 seconds
test loss 1.030 
Time of round: 10 seconds
test loss 1.025 
Time of round: 10 seconds
test loss 1.021 
Time of round: 10 seconds
test loss 1.016 
Time of round: 10 seconds
test loss 1.012 
Time of round: 10 seconds
test loss 1.008 
Time of round: 10 seconds
test loss 1.004 
Time of round: 10 seconds
test loss 1.000 
Time of round: 10 seconds
test loss 0.996 
Time of round: 10 seconds
test loss 0.992 
Time of round: 10 seconds
test loss 0.989 
Time of round: 10 seconds
test loss 0.985 
Time of rou

In [49]:
for t in range(30):
    train_epocs(model_s, epochs=15, lr=0.01)

Time of round: 10 seconds
test loss 0.960 
Time of round: 10 seconds
test loss 0.958 
Time of round: 10 seconds
test loss 0.954 
Time of round: 10 seconds
test loss 0.952 
Time of round: 10 seconds
test loss 0.949 
Time of round: 10 seconds
test loss 0.947 
Time of round: 10 seconds
test loss 0.944 
Time of round: 10 seconds
test loss 0.942 
Time of round: 10 seconds
test loss 0.939 
Time of round: 10 seconds
test loss 0.937 
Time of round: 10 seconds
test loss 0.935 
Time of round: 10 seconds
test loss 0.933 
Time of round: 10 seconds
test loss 0.931 
Time of round: 10 seconds
test loss 0.929 
Time of round: 10 seconds
test loss 0.926 
Time of round: 10 seconds
test loss 0.925 
Time of round: 10 seconds
test loss 0.923 
Time of round: 10 seconds
test loss 0.921 
Time of round: 10 seconds
test loss 0.919 
Time of round: 10 seconds
test loss 0.917 
Time of round: 10 seconds
test loss 0.915 
Time of round: 10 seconds
test loss 0.914 
Time of round: 10 seconds
test loss 0.912 
Time of rou

In [ ]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        return (U*V).sum(1) +  b_u  + b_v

In [ ]:
model = MF_bias(num_users, num_items, emb_size=100) #.cuda()

In [ ]:
train_epocs(model, epochs=10, lr=0.05, wd=1e-5)

12.913201332092285
9.158486366271973
4.393214225769043
1.1586315631866455
2.4618592262268066
3.745398998260498
2.4525110721588135
1.079404354095459
0.8143762350082397
1.3159016370773315
test loss 2.068 


In [ ]:
train_epocs(model, epochs=10, lr=0.01, wd=1e-5)

1.8920012712478638
1.3238822221755981
0.9342262148857117
0.7443917989730835
0.722421407699585
0.7777937054634094
0.8232869505882263
0.8219900727272034
0.7811893224716187
0.7271417379379272
test loss 0.798 


In [ ]:
train_epocs(model, epochs=10, lr=0.001, wd=1e-5)

0.6848835349082947
0.6707419157028198
0.6589115262031555
0.6492202877998352
0.6414564251899719
0.6353794932365417
0.6307392716407776
0.6272894144058228
0.6247975826263428
0.6230531334877014
test loss 0.751 
